In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import sklearn

# Data Preprocessing
# with open(·"./avg-test-result-in-paper", "r") as f:
with open("./performance-comparison-table", "r") as f:
    df = pd.read_csv(f, delim_whitespace=True, index_col=None, engine='python')

# print(df.loc[df['workload'] == 'yscbLoadA'] )
fss = df['file_system'].drop_duplicates().reset_index().drop(columns=['index'])['file_system']
workloads = df['workload'].drop_duplicates().reset_index().drop(columns=['index'])['workload']

for workload in workloads:
    for fs in fss:
        if fs == "SplitFS-YCSB" and (workload == "fileserver" or workload == "varmail"):
            continue
        if fs == "SplitFS-FILEBENCH" and workload == "ycsbLoadA":
            continue
        # print(workload, fs)
        peak_mems = df.loc[(df['file_system'] == fs) & (df['workload'] == workload)]['peak_mem(KiB)']
        average_mems = df.loc[(df['file_system'] == fs) & (df['workload'] == workload)]['avg_mem(KiB)']

        # remove the abnormal data using k-means
        mems = list(zip(peak_mems, average_mems))
        kmeans = KMeans(n_clusters=2, random_state=0).fit(mems)
        # obtain cluster-0 points
        cluster_0 = [mems[i] for i in range(len(mems)) if kmeans.labels_[i] == 0]
        # obtain cluster-1 points
        cluster_1 = [mems[i] for i in range(len(mems)) if kmeans.labels_[i] == 1]
        
        valid_mems = cluster_0 if len(cluster_0) > len(cluster_1) else cluster_1
        valid_mems = np.array(valid_mems)
        peak_mems = valid_mems[:, 0]
        average_mems = valid_mems[:, 1]
        peak_mem = np.mean(peak_mems)
        average_mem = np.mean(average_mems)

        print(workload, fs, int(peak_mem/1024), int(average_mem/1024))


fileserver NOVA 160 42
fileserver KILLER 181 39
fileserver SplitFS-FILEBENCH 210 105
fileserver EXT4-DAX 90 19
fileserver XFS-DAX 102 18
ycsbLoadA NOVA 142 65
ycsbLoadA KILLER 108 31
ycsbLoadA EXT4-DAX 116 35
ycsbLoadA XFS-DAX 190 72
ycsbLoadA SplitFS-YCSB 275 78
varmail NOVA 150 37
varmail KILLER 192 59
varmail SplitFS-FILEBENCH 874 427
varmail EXT4-DAX 852 335
varmail XFS-DAX 780 630
